In [1]:
import pandas as pd
from stage import fullPrint
from datetime import date
import yahoo_fin.stock_info as yf
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

In [2]:
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)

In [3]:
SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nyseList))
for ticker in nyseList:
    try:
        # df = yf.get_data(ticker, start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk")
        # df.to_pickle('stockData/nasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
import pickle
try:
    sectorDict = open('stockData/nyse.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

AHC not found
1/3333
2/3333
3/3333
4/3333
5/3333
6/3333
7/3333
8/3333
9/3333
ACM not found
10/3333
11/3333
12/3333
13/3333
14/3333
15/3333
16/3333
AKS not found
17/3333
18/3333
19/3333
20/3333
HKD not found
21/3333
22/3333
23/3333
24/3333
ARHPRA not found
25/3333
26/3333
27/3333
28/3333
TBB not found
29/3333
TBC not found
30/3333
31/3333
32/3333
AUO not found
33/3333
AVX not found
34/3333
35/3333
36/3333
37/3333
38/3333
39/3333
40/3333
41/3333
42/3333
43/3333
44/3333
ASGI not found
45/3333
46/3333
47/3333
48/3333
49/3333
50/3333
51/3333
ACW not found
52/3333
53/3333
ATU not found
54/3333
55/3333
56/3333
57/3333
58/3333
ADIN not found
59/3333
60/3333
61/3333
62/3333
63/3333
64/3333
65/3333
66/3333
ADSW not found
67/3333
68/3333
69/3333
AEFC not found
70/3333
71/3333
72/3333
73/3333
AET not found
74/3333
75/3333
76/3333
MGRB not found
77/3333
MGR not found
78/3333
AGACU not found
79/3333
80/3333
81/3333
82/3333
83/3333
84/3333
GAS not found
85/3333
86/3333
87/3333
GRO not found
88/3333
A

In [40]:
import numpy as np
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,8]/df.iat[index-1,8]
def peakCheck(df, index):
    if index == 0:
        return df.iat[index,3]
    else:
        return max(df.iat[index-1,13], df.iat[index,3])
def troughCheck(df, index):
    if index == 0:
        return df.iat[index,3]
    elif df.iat[index,13]==df.iat[index,3]:
        return df.iat[index,3]
    else:
        return min(df.iat[index-1,14], df.iat[index,3])
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,3]/df.iat[index-30,3]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,3]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,3])

In [41]:
from stage import fullPrint
import swifter
import yahoo_fin.stock_info as yf
import pandas as pd
spDf = yf.get_data('^GSPC', start_date="1995-01-02",end_date="2022-01-09", index_as_date = True, interval="1wk")
for ticker in nyseList:
    try:
        stockDf = pd.read_pickle('stockData/tickers/'+ticker+'.pkl')
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = stockDf['close'].rolling(window=104, closed='left').max().shift(1).fillna(0) 
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, 7] = calculatePeak(stockDf, numIndex, 260, 7)
            if numIndex < 104:
                stockDf.iat[numIndex, 13] = calculatePeak(stockDf, numIndex, 104, 13)
            stockDf.iat[stockDf.index.get_loc(index), stockDf.columns.get_loc('trough')] = troughCheck(stockDf,stockDf.index.get_loc(element.name))
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        print(ticker)
        stockDf.to_pickle('stockData/tickers/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 1410/1410 [00:00<00:00, 22380.80it/s]


ATEN


Pandas Apply: 100%|██████████| 1410/1410 [00:00<00:00, 21044.58it/s]


AIR


Pandas Apply: 100%|██████████| 1410/1410 [00:00<00:00, 19315.33it/s]


...

ZUO


Pandas Apply: 100%|██████████| 1410/1410 [00:00<00:00, 21694.20it/s]


ZWS


Pandas Apply: 100%|██████████| 1410/1410 [00:00<00:00, 21363.80it/s]


ZYME


In [37]:
fullPrint(pd.read_pickle('stockData/tickers/A.pkl')[['close','fiveYearHigh','peak']])

                 close  fiveYearHigh        peak
1999-11-15   28.880545     28.880545   28.880545
1999-11-22   29.461731     29.461731   29.461731
1999-11-29   31.831186     31.831186   31.831186
1999-12-06   32.010014     32.010014   32.010014
1999-12-13   32.859444     32.859444   32.859444
1999-12-20   35.586552     35.586552   35.586552
1999-12-27   55.302216     55.302216   55.302216
2000-01-03   46.494991     55.302216   55.302216
2000-01-10   48.909157     55.302216   55.302216
2000-01-17   49.177395     55.302216   55.302216
2000-01-24   48.685623     55.302216   55.302216
2000-01-31   54.542202     55.302216   55.302216
2000-02-07   53.916309     55.302216   55.302216
2000-02-14   67.060089     67.060089   67.060089
2000-02-21   77.342628     77.342628   77.342628
2000-02-28   77.253220     77.342628   77.342628
2000-03-06  101.573677    101.573677  101.573677
2000-03-13   86.641632    101.573677  101.573677
2000-03-20   85.836906    101.573677  101.573677
2000-03-27   74.3919